In [2]:
import numpy as np 
import cv2
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

In [3]:
def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True) 
    b.files = filedialog.askopenfilename(multiple=True) 
    print(b.files) 

fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

('C:/Users/lenovo/Downloads/ff4130071ef4044a7c1cd8f7709b5ff0.jpg',)


In [4]:
net = cv2.dnn.readNetFromCaffe('./model/colorization_deploy_v2.prototxt','./model/colorization_release_v2.caffemodel')
pts = np.load('./model/pts_in_hull.npy')


class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2,313,1,1)

net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1,313],2.606,dtype='float32')]

In [ ]:
files = fileselect.files
image = cv2.imread(files[0])
scaled = image.astype("float32")/255.0
lab = cv2.cvtColor(scaled,cv2.COLOR_BGR2LAB)


resized = cv2.resize(lab,(224,224))
L = cv2.split(lab)[0]
L -= 50


net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1,2,0))

ab = cv2.resize(ab, (image.shape[1],image.shape[0]))

L = cv2.split(lab)[0]
colorized = np.concatenate((L[:,:,np.newaxis], ab), axis=2)

colorized = cv2.cvtColor(colorized,cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized,0,1)
colorized = (255 * colorized).astype("uint8")

# image = cv2.resize(image, (400, 400)) 
# colorized = cv2.resize(colorized, (400, 400)) 
hasil = cv2.imwrite('./rubah.png', colorized)

cv2.imshow("Original",image)
cv2.imshow("Colorized",colorized)

cv2.waitKey(0)
print("Image written to file-system : ",hasil)


In [6]:

net = cv2.dnn.readNetFromCaffe('./model/colorization_deploy_v2.prototxt','./model/colorization_release_v2.caffemodel')

if not net.empty():
 print('Net loaded successfully\n')

print('Net contains:')
for t in net.getLayerTypes():
 print('\t%d layers of type %s' % (net.getLayersCount(t), t))




Net loaded successfully

Net contains:
	7 layers of type BatchNorm
	23 layers of type Convolution
	1 layers of type Deconvolution
	22 layers of type ReLU
	1 layers of type Scale
	1 layers of type Silence
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [75]:
layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 1, 224, 224])

layers_names = net.getLayerNames()

print('Net layers shapes:')
for l in range(len(layers_names)):
    in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
    print('Layer "%s" has %d input(s) and %d output(s)'
        % (layers_names[l], in_num, out_num))
    for i in range(in_num):
        print('\tinput #%d has shape' % i, in_shapes[l][i].flatten())

    for i in range(out_num):
        print('\toutput #%d has shape' % i, out_shapes[l][i].flatten())

SystemError: <method 'getLayersShapes' of 'cv2.dnn_Net' objects> returned NULL without setting an error

In [69]:
layers_names = net.getLayerNames()

print('Net layers shapes:')
for l in range(len(layers_names)):
    print('Layer "%s"'
        % (layers_names[l]))

Net layers shapes:
Layer "conv1_1"
Layer "relu1_1"
Layer "conv1_2"
Layer "relu1_2"
Layer "conv1_2norm"
Layer "conv2_1"
Layer "relu2_1"
Layer "conv2_2"
Layer "relu2_2"
Layer "conv2_2norm"
Layer "conv3_1"
Layer "relu3_1"
Layer "conv3_2"
Layer "relu3_2"
Layer "conv3_3"
Layer "relu3_3"
Layer "conv3_3norm"
Layer "conv4_1"
Layer "relu4_1"
Layer "conv4_2"
Layer "relu4_2"
Layer "conv4_3"
Layer "relu4_3"
Layer "conv4_3norm"
Layer "conv5_1"
Layer "relu5_1"
Layer "conv5_2"
Layer "relu5_2"
Layer "conv5_3"
Layer "relu5_3"
Layer "conv5_3norm"
Layer "conv6_1"
Layer "relu6_1"
Layer "conv6_2"
Layer "relu6_2"
Layer "conv6_3"
Layer "relu6_3"
Layer "conv6_3norm"
Layer "conv7_1"
Layer "relu7_1"
Layer "conv7_2"
Layer "relu7_2"
Layer "conv7_3"
Layer "relu7_3"
Layer "conv7_3norm"
Layer "conv8_1"
Layer "relu8_1"
Layer "conv8_2"
Layer "relu8_2"
Layer "conv8_3"
Layer "relu8_3"
Layer "conv8_313"
Layer "conv8_313_rh"
Layer "class8_313_rh"
Layer "class8_ab"
Layer "Silence"


In [21]:
files = fileselect.files
image = cv2.imread(files[0], cv2.IMREAD_COLOR)
tensor = cv2.dnn.blobFromImage(image, 1.0, (224, 224),(104, 117, 123), False, False)


In [22]:
tensor = cv2.dnn.blobFromImages([image, image], 1.0, (224, 224),(104, 117, 123), False, True);


In [23]:
net.setInput(tensor);
prob = net.forward();

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wvn_it83\opencv\modules\dnn\src\layers\convolution_layer.cpp:381: error: (-215:Assertion failed) ngroups > 0 && inpCn % ngroups == 0 && outCn % ngroups == 0 in function 'cv::dnn::ConvolutionLayerImpl::getMemoryShapes'
